In [ ]:
%pip install -qU langchain langchain_community

# Local vector store via Chroma
%pip install -qU langchain_chroma

# Local inference and embeddings via Ollama
%pip install -qU langchain_ollama

# Web Loader
%pip install -qU beautifulsoup4

In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=20)

Compile data together

In [2]:
import pandas as pd

entries = pd.read_csv("processed.csv")

In [12]:
notes_compiled = ""


#full_date,date,weekday,time,mood,activities,note_title,note

for idx, entry in entries.iterrows():
    notes_compiled += f"""
    { str(entry['note']) if str(entry['note']) != "nan" else ""}
    """
    

In [13]:
notes_compiled = notes_compiled.replace("<br>", ".")
notes_compiled = notes_compiled.replace("</br>", ".")

notes_compiled = notes_compiled.replace("<b>", ".")
notes_compiled = notes_compiled.replace("</b>", ".")


In [14]:
with open("Output.txt", "w") as text_file:
    text_file.write(notes_compiled)

In [15]:
all_splits = text_splitter.split_text(notes_compiled)

In [16]:
len(all_splits)

4622

In [17]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

local_embeddings = OllamaEmbeddings(model="nomic-embed-text")

vectorstore = Chroma.from_texts(texts=all_splits, embedding=local_embeddings)

In [23]:
question = "who am i as a person and what do i value?"
docs = vectorstore.similarity_search(question, k=25)
print(len(docs))
print(docs[0])

25
page_content='We are currently in quiet time and I wanted to use this opportunity to reflect on who I am and the things I struggle with...My name is Eyiaraoluwa oladipo and I am 19 years old. ..My values ..I am finding it hard to pinpoint my values but part of me is also wondering whether this is the best use of my time, but in order to have fruitful relationships with other people, I need to have and know my values, and it can help me understand better why I act the way I do..One thing I value is dependability, and'


In [26]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="llama3.1:8b",
)

In [27]:
from IPython.display import Markdown, display

response_message = model.invoke(
    "how many letter rs are in strawberry"
)

# Compile the output to markdown
markdown_output =response_message.content

# Print or display the markdown output
display(Markdown(markdown_output))

There is 1 "R" in the word "strawberry".

In [58]:
class ChatMemory:
    """Handles the memory of the conversation history."""
    def __init__(self):
        self.history = []
        
    def add_to_memory(self, user_input, assistant_output):
        """Add a new exchange (user question and assistant answer) to memory."""
        self.history.append(f"User: {user_input}")
        self.history.append(f"Assistant: {assistant_output}")
        
    def get_context(self):
        """Return the full context (conversation history) as a single string."""
        return "\n".join(self.history)



In [44]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

RAG_TEMPLATE = """
You are a helpful assistant who keeps track of all past exchanges. Here is your conversation history
<Conversation History>
{history}
</Conversation History>

Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. These are some entries at different dates that give insight into your state of mind, memories and moods. Use direct references when necessary
Be as specific and thorough as possible
<context>
{context}
</context>

Answer the following question:

{question}"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

chat_memory = ChatMemory()

chain = (
    RunnablePassthrough.assign(context=lambda input: format_docs(input["context"]))
    | rag_prompt
    | model
    | StrOutputParser()
)


In [45]:
def ask_question(question, previous_docs=None):
    # Update this to use all context?
    context = chat_memory.get_context() if previous_docs is None else previous_docs
    
    # Get documents related to the current question
    docs = vectorstore.similarity_search(question, k=25)
    
    markdown_output = chain.invoke({"history": context, "context": docs, "question": question})
    
    # Add to the chat memory
    chat_memory.add_to_memory(question, markdown_output)
    
    # Return the response
    return markdown_output


In [ ]:
question = "Tell me about my relationship with myself?"
response = ask_question(question)

display(Markdown(response))

print(chat_memory.get_context())

follow_up_question = "can you tell me more about my concerns?"

follow_up_response = ask_question(follow_up_question, previous_docs=response)
display(Markdown(follow_up_response))

print(chat_memory.get_context())


# You can continue the conversation with more questions
another_question = "How can I work on these feelings to be closer with myself?"
another_response = ask_question(another_question, previous_docs=follow_up_response)
display(Markdown(another_response))

print(chat_memory.get_context())
